1.True false
    a.False
    b.False 
    c.False
    d.False
    e.False
    f.False
    g.False
    h.False -->it’s only outperforms for sure when there is a class imbalance, not always balanced
    i.True
    j.False
    
2.In what scenarios, RandomSearchCV would be preferred instead of GridSearchCV? Be specific.
    The only difference is that RSCV picks random combinations. You should use RSCV when you have big data sets or a lot of hyper parameters you are trying to tune.
    
3.How does stacking work?
    Its splits the dataset into training and testing, trains base learners on the training dataset, applies the base learners on the testing to make predictions and then uses the predictions as inputs to the actual responses as outputs to train a higher-level learner. It’s important that the predictions from the base learners are not correlated with each other.

4.If you have trained three different models on the exact same training dataset, and they all achieve a 93% F1-score on the exact same dataset. Is there any chance that you can combine these three models to get better results? If so, how? If not, why? Be specific.
    There is an ensemble method calling voting and it combines prediction of models by voting. You can combine the models and try the voting method but it’s not certain it will be able to improve the overall F1 score because of how similar the models are performing.  

5.If your XGBoost model overfits the data (probably because the number of boosted trees is large), what hyper-parameter you need to tune? Be specific.
    n_estimators -->May improve performance
    max_depth --> decreasing prevents over fitting
    Gamma --> increasing prevents over fitting
    Min_child_weight  Increasing prevents overfitting
    
6.Which of the following algorithm is NOT an ensemble learning algorithm?
    a.Support Vector machine
    
7.How does XGBoost with random forest as base learner work? Be specific.
    It runs when you tune the num_parallel_trees from its default variable. It allows you to choose the number of parallel trees 
    you want to run during each boosting round. Then the trees get combined to create a Random Forest for each round!
    
8.How do we select the best hyper-parameter combination for a given model?
    b. Performance on the validation dataset
    
9.Picture
    a.He should select 400 trees because we tune the training data set and that is the lowest number of trees that gives us the highest F-1 score.
    b.The reason for the overfitting could be tuning factors like number of estimators and depth of tree. Have too many of each can cause the model to overfit, 
    but there are others as well that can be tuned. 
    
10.What is the difference between homogeneous and heterogeneous ensembles? Be specific.
    Heterogeneous ensemble methods have different algorithms for base learners and Homogeneous ensemble methods is where the same algorithms are used for all the base learners.
    
11.Random forest is an example of
    d: a and c --> Homo and Bagging
    
12.A junior data scientist is working on building a classification framework that can help a small local credit union to identify fraudulent transactions. 
The data scientist collects a very small dataset with 100 observations, from which 4 observations are labelled as fraudulent, to start building and tuning the model. 
As part of the tuning process, the data scientist considers using the GridSearch function from scikit-learn with cv = 5. Explain why this approach is a good idea or not. Be specific.
    This is not a good idea because when you create the splits, at least one of them won’t have a fraudulent transaction included. 
    What if all 4 are in the same one and the other 4 splits don’t have one? I don’t think this is a good approach to do when the number of 
    fraudulent transactions can’t be evenly split into the folds. 
    
13.The typical hyper-parameters in a XGBoost model that data science practitioners tune are:
    a.Max_depth because it causes more relationships to be created making the model more complex, Subsample because decreasing prevents overfitting 
        as with colsample_bytree and learning_rate.
    b.n_estimators! By increasing it may improve results.
    
14.  In tree-based models such as random forest, gradient boosting, and XGBoost,
how does a data scientist optimize the number of trees in a manually fashion? Be specific.
    They can set up their desired model and start by running it with a small amount of trees (say 10-50). Use an appropriate performance measurement to evaluate performance. 
    Once you have a base performance, increase the number of trees until you hit a point where the performance no longer improves. 
    Many times, the optimal number of trees isn’t a clean number like 500, it could be 459, but getting to a number like 550 or 
    560 can still get you to a umber close to the best performance!
    
15.A Machine Learning Specialist is assigned to a team that is responsible for tuning an XGBoost model to ensure that it performs correctly on test data. 
However, when dealing with unknown data, this does not operate as planned. The following table summarizes the existing hyper-parameters:
    a.Regression because the objective is the squared error!
    b.There are 2000 estimators, and the depth is 30, both are very significant amounts!
    
16.How can overfitting and underfitting be addressed through hyper-parameter tuning in random forest? That is, what hyper-parameters would you tune 
to address overfitting and underfitting in random forest? Be specific.
    Most common parameters: n_estimators, min_samples_split, min_samples_leaf, max_depth. 
    n_estimators finding a good number of trees, so the model isn’t under or overfitting and getting a good performance overall.
    min_samples_split helps prevent the model from branching out too much and following noise, it helps prevent overfitting.
    mine_samples_leaf forces more observations to be at the leaf’s getting a bigger picture of the data, increasing will help over fitting.
    max_depth to keep the model from being too complex and overfitting but deep enough so the important patterns are analyzed (not underfitting).


In [1]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 4.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 16.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xgboost

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 4.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [20]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna

from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold, KFold
from sklearn.metrics import classification_report, confusion_matrix, make_scorer, mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE, RFECV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error  


s3= boto3.resource('s3')
bucket_name= 'morgan-gant-data448-bucket'
bucket= s3.Bucket(bucket_name)

file_key= 'Customers.csv'

bucket_object= bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the datefile
customers = pd.read_csv(file_content_stream)
customers.head()

,CustomerID,Gender,Age,Annual Income ($),Spending Score (1-100),Profession,Work Experience,Family Size
0,1,Male,19,15000,39,Healthcare,1,4
1,2,Male,21,35000,81,Engineer,3,3
2,3,Female,20,86000,6,Engineer,1,1
3,4,Female,23,59000,77,Lawyer,0,2
4,5,Female,31,38000,40,Entertainment,2,6


In [21]:
customers= customers.drop(columns='CustomerID')
customers.head()

,Gender,Age,Annual Income ($),Spending Score (1-100),Profession,Work Experience,Family Size
0,Male,19,15000,39,Healthcare,1,4
1,Male,21,35000,81,Engineer,3,3
2,Female,20,86000,6,Engineer,1,1
3,Female,23,59000,77,Lawyer,0,2
4,Female,31,38000,40,Entertainment,2,6


In [22]:
#Replacing na values in profession
#Identifying categorical variables
categorical_vars = customers.select_dtypes(include = 'object').columns.to_list()
print(categorical_vars)

frequent_values = customers[categorical_vars].mode().iloc[0].to_dict()
print(frequent_values)

## Filling missing values with most frequent labels
customers = customers.fillna(value = frequent_values)

['Gender', 'Profession']
{'Gender': 'Female', 'Profession': 'Artist'}


In [23]:
#Identifying numeric variables
numeric_vars = customers.select_dtypes(exclude = 'O').columns.to_list()
print(numeric_vars)

#Choosing median because the data isn't normal
#Compute the median values of the numerical variables and store in a dictionary
median_values = customers[numeric_vars].median().to_dict()
median_values

#Let’s replace missing data with the median
customers = customers.fillna(value = median_values)

['Age', 'Annual Income ($)', 'Spending Score (1-100)', 'Work Experience', 'Family Size']


In [24]:
#variables 1 and 2
#changing gender to dummy variables
customers= pd.concat([customers.drop(columns=['Gender', 'Profession'], axis=1), pd.get_dummies(customers[['Gender', 'Profession']])], axis=1)

In [25]:
#variables 3,4,5
#Engineering interactions based on feature importance
customers['interaction_3']= customers['Age'] * customers['Annual Income ($)']
customers['interaction_4']= customers['Age'] * customers['Work Experience']
customers['interaction_5']= customers['Annual Income ($)'] * customers['Work Experience']

In [26]:
#variable 6: x^2
customers['Annual Income^2']= np.square(customers['Annual Income ($)'])

In [27]:
#variables 7 and 8
#engineering features from the decisoon tree model
customers['interaction_6']=np.where(((customers['interaction_3'] <= 17141355) &
                                   (customers['interaction_5'] <= 1016846.5) &
                                    (customers['Annual Income^2'] <= 24427815936)), 1, 0)
                                 
customers['interaction_7']=np.where(((customers['Annual Income ($)'] <= 189472) &
                                   (customers['interaction_5'] <= 1016846.5) &
                                (customers['Annual Income ($)'] <= 187714)), 1, 0)
        


In [89]:
#defining input and target variables

x_train= train[['Annual Income^2', 'Annual Income ($)', 'interaction_3', 'interaction_5', 'Age', 'interaction_4', 'Family Size']]
x_test= test[['Annual Income^2', 'Annual Income ($)', 'interaction_3', 'interaction_5', 'Age', 'interaction_4', 'Family Size']]
y_train= train[['Spending Score (1-100)']]

In [28]:
#Defining input and target variables
x= customers[['Annual Income^2', 'Annual Income ($)', 'interaction_3', 'interaction_5', 'Age', 'interaction_4', 'Family Size']]
y= customers[['Spending Score (1-100)']]

#splitting the data
x_train, x_test, y_train, y_test= train_test_split(x,y, test_size= .2, stratify=y)

Random Forest

In [37]:
class Objective:
    def __init__(self, seed):
        self.seed = seed
        
    def __call__(self, trial):
    ## Parameters to be evaluated
        params = dict(n_estimators = trial.suggest_int('n_estimators', 100, 2000),
            min_samples_split = trial.suggest_int('min_samples_split', 5, 30),
            min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 30),
            max_depth = trial.suggest_int('max_depth', 2, 10)
            )
        scores= list()
         
        #Running cross validation
        skf = KFold(n_splits = 3, shuffle = True, random_state = self.seed)
       
        for train_idx, valid_idx in skf.split(x_train, y_train):
            x_train_1, x_valid_1 = x_train.iloc[train_idx], x_train.iloc[valid_idx]
            y_train_1 , y_valid_1 = y_train.iloc[train_idx] , y_train.iloc[valid_idx]
         
            rf_md = RandomForestRegressor(**params).fit(x_train_1, y_train_1)
         
            preds_valid = rf_md.predict(x_valid_1)
              
            rmse = np.sqrt(mean_squared_error(y_valid_1, preds_valid))    
            
            scores.append(rmse)
                     
        return np.mean(scores)

In [38]:
SEED= 42
N_TRIALS= 50

study_1= optuna.create_study(direction= 'minimize')
study_1.optimize(Objective(SEED), n_trials= N_TRIALS)

[I 2023-04-05 01:14:27,727] A new study created in memory with name: no-name-f9276d1e-8f68-4a93-a57e-f6883e92345d
/tmp/ipykernel_16017/1802527724.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_md = RandomForestRegressor(**params).fit(x_train_1, y_train_1)
/tmp/ipykernel_16017/1802527724.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_md = RandomForestRegressor(**params).fit(x_train_1, y_train_1)
/tmp/ipykernel_16017/1802527724.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_md = RandomForestRegressor(**params).fit(x_train_1, y_train_1)
[I 2023-04-05 01:14:30,754] Trial 0 finished with value: 28.11044250308107 and parameters: {'n_estim

In [40]:
print(study_1.best_trial.params)

{'n_estimators': 1912, 'min_samples_split': 24, 'min_samples_leaf': 25, 'max_depth': 2}


In [122]:
rf_md = RandomForestRegressor(**study_1.best_trial.params).fit(x_train, y_train)
         
rf_pred = rf_md.predict(x_test)
              
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))

print('RF rmse:', rf_rmse)

/tmp/ipykernel_16017/3434117746.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_md = RandomForestRegressor(**study_1.best_trial.params).fit(x_train, y_train)


RF rmse: 27.851389523376305


XGBoost

In [41]:
from xgboost import XGBRegressor

In [110]:
class Objective:
    def __init__(self, seed):
        self.seed = seed
        
    def __call__(self, trial):
    ## Parameters to be evaluated
        params = dict(n_estimators = trial.suggest_int('n_estimators', 100, 2000),
            min_child_weight = trial.suggest_int('min_child_weight', 5, 7, 10),
            learning_rate = trial.suggest_uniform('learning_rate', .001, .1),
            gamma= trial.suggest_float('gamma', .1, 10.0),
            subsample= trial.suggest_int('subsample', .8, 1),
            colsample_bytree= trial.suggest_int('colsample_bytree', .8, 1),
            max_depth = trial.suggest_int('max_depth', 2, 10)
            )
         
        scores= list()
         
        #Running cross validation
        skf = KFold(n_splits = 3, shuffle = True, random_state = self.seed)
       
        for train_idx, valid_idx in skf.split(x_train, y_train):
            x_train_1, x_valid_1 = x_train.iloc[train_idx], x_train.iloc[valid_idx]
            y_train_1 , y_valid_1 = y_train.iloc[train_idx] , y_train.iloc[valid_idx]
         
            xgb_md = XGBRegressor(**params).fit(x_train_1, y_train_1)
         
            preds_valid = xgb_md.predict(x_valid_1)
              
            rmse = np.sqrt(mean_squared_error(y_valid_1, preds_valid))    
            
            scores.append(rmse)
                     
        return np.mean(scores)

In [112]:
SEED= 42
N_TRIALS= 50

study_2= optuna.create_study(direction= 'minimize')
study_2.optimize(Objective(SEED), n_trials= N_TRIALS)

[I 2023-04-05 02:07:11,186] A new study created in memory with name: no-name-ae29781f-3bdf-4a34-9d2b-bb88317da1c7
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/optuna/distributions.py:716: UserWarning: The distribution is specified by [5, 7] and step=10, but the range is not divisible by `step`. It will be replaced by [5, 5].
  warnings.warn(
/tmp/ipykernel_16017/3886774883.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  learning_rate = trial.suggest_uniform('learning_rate', .001, .1),
[I 2023-04-05 02:07:12,593] Trial 0 finished with value: 28.541649850310986 and parameters: {'n_estimators': 664, 'min_child_weight': 5, 'learning_rate': 0.023099492019152974, 'gamma': 4.821639534588421, 'subsample': 1, 'colsample_bytree': 0, 'max_depth': 2}. Best is trial 0 with value: 28.541649850310986.
/home

In [114]:
print(study_2.best_trial.params)

{'n_estimators': 184, 'min_child_weight': 5, 'learning_rate': 0.022592896278907232, 'gamma': 6.344984225203609, 'subsample': 1, 'colsample_bytree': 1, 'max_depth': 2}


In [121]:
xgb_md = XGBRegressor(**study_2.best_trial.params).fit(x_train, y_train)
         
xgb_pred = xgb_md.predict(x_test)
              
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_pred))

print('XGB rmse:', xgb_rmse)

XGB rmse: 27.888274384328444


Gradient Boost

In [84]:
from sklearn.ensemble import GradientBoostingRegressor

In [125]:
class Objective:
    def __init__(self, seed):
        self.seed = seed
        
    def __call__(self, trial):
    ## Parameters to be evaluated
        params = dict(n_estimators = trial.suggest_int('n_estimators', 100, 2000),
            min_samples_split = trial.suggest_int('min_samples_split', 5, 30),
            min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 30),
            max_depth = trial.suggest_int('max_depth', 2, 10),
            learning_rate= trial.suggest_uniform('learning_rate', .001,.01)
            )
        scores= list()
         
        #Running cross validation
        skf = KFold(n_splits = 3, shuffle = True, random_state = self.seed)
       
        for train_idx, valid_idx in skf.split(x_train, y_train):
            x_train_1, x_valid_1 = x_train.iloc[train_idx], x_train.iloc[valid_idx]
            y_train_1 , y_valid_1 = y_train.iloc[train_idx] , y_train.iloc[valid_idx]
         
            gb_md = GradientBoostingRegressor(**params).fit(x_train_1, y_train_1)
         
            preds_valid = gb_md.predict(x_valid_1)
              
            rmse = np.sqrt(mean_squared_error(y_valid_1, preds_valid))    
            
            scores.append(rmse)
                     
        return np.mean(scores)

In [126]:
SEED= 42
N_TRIALS= 50

study_3= optuna.create_study(direction= 'minimize')
study_3.optimize(Objective(SEED), n_trials= N_TRIALS)

[I 2023-04-05 02:22:06,749] A new study created in memory with name: no-name-651ca19f-a5ba-4396-b1e8-98b6cbc063c5
/tmp/ipykernel_16017/3891759447.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  learning_rate= trial.suggest_uniform('learning_rate', .001,.01)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=Tr

In [127]:
print(study_3.best_trial.params)

{'n_estimators': 141, 'min_samples_split': 24, 'min_samples_leaf': 13, 'max_depth': 3, 'learning_rate': 0.0013348644069811227}


In [128]:
gb_md = GradientBoostingRegressor(**study_3.best_trial.params).fit(x_train, y_train)
         
gb_pred = gb_md.predict(x_test)
              
gb_rmse = np.sqrt(mean_squared_error(y_test, gb_pred))

print('GB rmse:', gb_rmse)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GB rmse: 27.913332011464878


Ensemble

In [129]:
en_pred= (rf_pred + xgb_pred + gb_pred)/3

en_rmse= np.sqrt(mean_squared_error(y_test, en_pred))

print('Ensemble rmse', en_rmse)

print('From Above:')
print('GB rmse:', gb_rmse)
print('XGB rmse:', xgb_rmse)
print('RF rmse:', rf_rmse)

Ensemble rmse 27.856126014068778
From Above:
GB rmse: 27.913332011464878
XGB rmse: 27.888274384328444
RF rmse: 27.851389523376305


In [ ]:
#When considering rmse, you want it to be low. Keeping this in mind, to predict
#on spending score, I would want to use the Random Forest model, which has a RMSE score of 27.85138
#All scores were very similar though!

In [ ]:
#I wanted to implement Adaboost so there was a little more difference in my
#models but I kept running into an error and I got mad and picked GB instead:)